## 第8章 分析句子结构

前面的章节重点关注词：如何识别它们，分析它们的结构，分配给他们词汇类别，以及获得它们的含义。我们还看到了如何识别词序列或 `n-grams` 的模式。然而，这些方法只触碰到管理句子的复杂约束的表面。我们需要一种方法处理自然语言中显著的歧义。我们还需要能够应对这样一个事实：句子有无限的可能，而我们只能写有限的程序来分析其结构和发现它们的含义。

本章的目的是要回答下列问题： 
1. 我们如何使用形式化语法来描述无限的句子集合的结构？ 
2. 我们如何使用句法树来表示句子结构？ 
3. 语法分析器如何分析一个句子并自动构建语法树？ 

一路上，我们将覆盖英语语法的基础，并看到句子含义有系统化的一面，只要我们确定了句子结构，将更容易捕捉。

### 8.1 一些语法困境
#### 语言数据和无限可能性

文法的目的是给出一个明确的语言描述。

在这一章中，我们将采取“生成文法”的形式化框架，其中一种“语言”被认为是所有 合乎文法的句子的巨大集合，一个文法是一个形式化符号，可用于“产生“这个集合的成员。 文法使用`S → S and S` 形式的递归产生式，我们将在 8.3节探讨。第 10章中，我们会将其扩展到从它的组成部分的意思自动建立一个句子的意思。

#### 普遍存在的歧义

让我们仔细看看短语 `I shot an elephant in my pajamas` 中的歧义。首先，我们需要定义一个简单的文法：

In [2]:
import nltk
groucho_grammar = nltk.CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

In [4]:
# 这个文法允许以两种方式分析句子，取决于介词短语 in my pajamas 是描述大象还是枪击事件。
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
trees = parser.parse(sent)
for tree in trees:
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


本章介绍文法和分析，以形式化可计算的方法调查和建模我们一直在讨论的语言现象。 正如我们所看到的，词序列中符合语法规则的和不符合语法规则的模式相对于短语结构和依赖是可以被理解的。我们可以开发使用文法和分析的这些结构的形式化模型。与以前一样，一个重要的动机是自然语言理解。当我们能可靠地识别一个文本所包含的语言结构时，我们从中可以获得多少文本的含义？一个程序通读了一个文本，它能否足够“理解”它，来回答一些简单的关于“发生了什么事”或“谁对谁做了什么”的问题？还像以前一样，我们将开发简单的程序来处理已注释的语料库，并执行有用的任务。

### 8.2 文法有什么用？
#### 超越`n-grams`

并列结构：如果`v1`和`v2`都是文法类型`X`的短语，那么 `v1 and v2` 也是`X` 类型的短语。

成分结构基于对词与其他词结合在一起形成单元的观察。一个词序列形成这样一个单元的证据是它是可替代的——也就是说，在一个符合语法规则的句子中的词序列可以被一个更小的序列替代而不会导致句子不符合语法规则。

### 8.3 上下文无关文法
#### 一种简单的文法

首先，让我们看一个简单的上下文无关文法（`context-free grammar，CFG`）。按照惯例， 第一条生产式的左端是文法的开始符号，通常是 `S`，所有符合语法规则的树都必须有这个符号作为它们的根标签。NLTK 中，上下文无关文法定义在`nltk.grammar` 模块。在例 8-1 中，我们定义了文法，并显示如何分析一个简单的符合文法的句子。

In [6]:
# 例8-1. 上下文无关文法的例子。
grammar1 = nltk.CFG.fromstring("""
    S -> NP VP
    VP -> V NP | V NP PP
    PP -> P NP
    V -> "saw" | "ate" | "walked"
    NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
    Det -> "a" | "an" | "the" | "my"
    N -> "man" | "dog" | "cat" | "telescope" | "park"
    P -> "in" | "on" | "by" | "with"
""")

sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)

(S (NP Mary) (VP (V saw) (NP Bob)))


例 8-1 中的文法包含涉及各种句法类型的产生式，如表 8-1 中所列出的。在这里使用**递归下降分析器**也可以通过图形界面查看，如图 8-3 所示；我们在 8.4节中将更详细讨论这个分析器。
![8-3](./imgs/8-3.jpg)

![list8-1](./imgs/list8-1.jpg)

产生式如`VP -> V NP | V NP PP` 右侧脱节了，中间显示`|`，这是两个产生式 `VP -> V NP` 和 `VP -> V NP PP` 的缩写。

如果我们使用例 8-1 显示的文法分析句子`The dog saw a man in the park`，我们会得到两棵树，类似与我们在(3)中看到的：
![cfg](./imgs/cfg.jpg)

由**于这句话的两棵树都符合我们的文法规则，这句话被称为结构上有歧义**。这个歧义问题被称为**介词短语附着歧义**，正如我们在本章前面看到的。正如你可能还记得，这是一个附着歧义，因为 `PP in the park`需要附着在树中两个位置中的一个：要么是`VP` 的孩子要么是 `NP`的孩子。当 `PP`附着在`VP` 上，合适的解释是：看到公园里发生的事情。然而，如果`PP` 附着在`NP` 上，意思就是：在公园里的一个人，看到（狗）的主语可能已经坐在公寓的阳台上俯瞰公园。

#### 写你自己的文法

如果你有兴趣尝试写上下文无关文法`CFG`，你会发现在一个文本文件`mygrammar.cfg`中创建和编辑你的语法会很有帮助。然后，你可以加载它到NLTK 中，并按如下方式用它进行分析：

In [7]:
grammar1 = nltk.data.load('file:mygrammar.cfg')
sent = "Mary saw Bob".split()
rd_parser = nltk.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
    print(tree)

LookupError: 
**********************************************************************
  Resource [93me:[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('e:')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93m/e:/codes/vscode_home/pythonNLP/mygrammar.cfg[0m

  Searched in:
    - ''
**********************************************************************


确保你的文件名后缀为`.cfg`，并且字符串`'file:mygrammar.cfg'`中间没有空格符。如果
命令`print tree` 没有产生任何输出，这可能是因为你的句子`sent` 并不符号你的文法。遇到这种情况可以将分析器的跟踪设置打开`rd_parser = nltk.RecursiveDescentParser(grammar1, trace=2)`。你还可以查看当前使用的文法中的产生式，使用命令 `for p in grammar1.productions(): print p`。 当你写`CFG` 在 NLTK中分析时，你不能将文法类型与词汇项目一起写在同一个产生式的右侧。因此，产生式`PP -> 'of' NP` 是不允许的。另外，你不得在产生式右侧仿制多个词的词汇项。因此，不能写成 `NP -> 'New York'`，而要写成类似` NP -> 'New_York'`这样的。

#### 句法结构中的递归

**一个文法被认为是递归的，如果文法类型出现在产生式左侧也出现在右侧**，如例 8-2所
示。产生式`Nom -> Adj Nom`（其中`Nom`是名词性的类别）包含 `Nom`类型的直接递归， 而`S` 上的间接递归来自于两个产生式的组合：`S -> NP VP` 与`VP -> V S`。

In [9]:
# 例8-2. 递归的上下文无关文法。
grammar2 = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det Nom | PropN
    Nom -> Adj Nom | N
    VP -> V Adj | V NP | V S | V NP PP
    PP -> P NP
    PropN -> 'Buster' | 'Chatterer' | 'Joe'
    Det -> 'the' | 'a'
    N -> 'bear' | 'squirrel' | 'tree' | 'fish' | 'log'
    Adj -> 'angry' | 'frightened' | 'little' | 'tall'
    V -> 'chased' | 'saw' | 'said' | 'thought' | 'was' | 'put'
    P -> 'on'
""")

要看递归如何从这个语法产生，思考下面的树。（10a）包括嵌套的名词短语，而（10b） 包含嵌套的句子。
![10a](./imgs/10a.jpg)

![10b](./imgs/10b.jpg)

我们在这里只演示了两个层次的递归，递归深度是没有限制的。你可以尝试分析更深层次嵌套结构的句子。请注意，`RecursiveDescentParser`是无法处理形如 `X -> X Y`的左递归产生式；我们将在第8.4节谈及这个。

### 8.4 上下文无关文法分析

**分析器**根据文法产生式处理输入的句子，并建立一个或多个符合文法的组成结构。**文法是一个格式良好的声明规范——它实际上只是一个字符串**，而不是程序。**分析器是文法的解释程序**。它搜索符合文法的所有树的空间找出一棵边缘有所需句子的树。分析器允许使用一组测试句子评估一个文法，帮助语言学家发现在他们的文法分析中存在的错误。分析器可以作为心理语言处理模型，帮助解释人类处理某些句法结构的困难。许多自然语言应用程序都在某种程度涉及文法分析；例如：我们会期望自然语言问答系统对提交的问题首先进行文法分析。

在本节中，我们将看到两个简单的分析算法，**一种自上而下的方法称为递归下降分析**， **一种自下而上的方法称为移进-归约分析**。我们也将看到一些更复杂的算法，一种带自下而上过滤的自上而下的方法称为左角落分析；一种动态规划技术称为图表分析。

#### 递归下降分析

一种最简单的分析器将一个文法作为如何将一个高层次的目标分解成几个低层次的子目标的规范来解释。顶层的目标是找到一个`S`。`S→NP VP` 生产式允许分析器替换这个目标为两个子目标：找到一个`NP`，然后找到一个 `VP`。每个这些子目标都可以再次被子目标的子目标替代，使用左侧有`NP` 和`VP` 的产生式。最终，这种扩张过程达到子目标，如：找到词 `telescope`。这样的子目标可以直接与输入序列比较，如果下一个单词匹配就取得成功。如果没有匹配，分析器必须备份，并尝试其它选择。

递归下降分析器在上述过程中建立分析树。带着最初的目标（找到一个`S`），创建 `S`根节点。随着上述过程使用文法的产生式递归扩展，分析树不断向下延伸（故名为递归下降）。我们可以在图形化示范`nltk.app.rdparser()`中看到这个过程。执行此分析器的六个阶段， 如图 8-4 所示。
![8-4](./imgs/8-4.jpg)

在这个过程中，分析器往往被迫在多种可能的产生式中选择。例如：从第3步到第4步，它试图找到左侧有`N` 的产生式。第一个是`N → man`。当这不起作用时就回溯，按顺序尝试其他左侧有`N` 的产生式，直到它得到 `N → dog`，与输入句子中的下一个词相匹配。一段时间以后，如第5 步所示，它发现了一个完整的分析树。这是一个涵盖了整个句子的树，没有任何悬着的边。发现了分析树后我们可以让分析器寻找其它额外的分析树。它会再次回溯和探索选择其它产生式，以免漏掉任何一个产生分析树的情况。

In [10]:
nltk.app.rdparser()

In [11]:
# NLTK 提供了一个递归下降分析器：
rd_parser = nltk.RecursiveDescentParser(grammar1)
sent = "Mary saw a dog".split()
for t in rd_parser.parse(sent):
    print(t)

(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


递归下降分析有三个主要的缺点。首先，左递归产生式，如：`NP -> NP PP`，会进入死循环。第二，分析器浪费了很多时间处理不符合输入句子的词和结构。第三，回溯过程中可能会丢弃分析过的成分，它们将需要在之后再次重建。例如：从`VP -> V NP`上回溯将放弃为`NP` 创建的子树。如果分析器之后处理 `VP -> V NP PP`，那么`NP` 子树必须重新创建。**递归下降分析是一种自上而下分析**。自上而下分析器在检查输入之前先使用文法预测输入将是什么！然而，由于输入对分析器一直是可用的，从一开始就考虑输入的句子会是更明智的做法。这种方法被称为**自下而上分析**，在下一节中我们将看到一个例子。


#### 移进-归约分析

**一种简单的自下而上分析器是移进-归约分析器**。与所有自下而上的分析器一样，移进-归约分析器尝试找到对应文法生产式右侧的词和短语的序列，用左侧的替换它们，直到整个句子归约为一个S。 移位-规约分析器反复将下一个输入词推到堆栈（见 4.1节），这是移位操作。如果堆栈上的前n 项，匹配一些产生式的右侧的n 个项目，那么就把它们弹出栈，并把产生式左边的项目压入栈。**这种替换前n 项为一项的操作就是规约操作**。**此操作只适用于堆栈的顶部**；规约栈中的项目必须在后面的项目被压入栈之前做。当所有的输入都使用过，堆栈中只剩余一个项目，也就是一颗分析树作为它的根的S节点时，分析器完成。移位-规约分析器通过上述过程建立一颗分析树。每次弹出堆栈 n 个项目，它就将它们组合成部分的分析树，然后将 这压回推栈。我们可以使用图形化示范 `nltk.app.srparser()`看到移位-规约分析算法步骤。 执行此分析器的六个阶段如图 8-7 所示。
![8-7](./imgs/8-7.jpg)

In [12]:
nltk.app.srparser()

NLTK中提供了`ShiftReduceParser()`，移进-归约分析器的一个简单的实现。这个分析器不执行任何回溯，所以它不能保证一定能找到一个文本的解析，即使确实存在一个这样的解析。此外，它最多只会找到一个解析，即使有多个解析存在。我们可以提供一个可选的 `trace` 参数，控制分析器报告它分析一个文本的步骤的繁琐程度。

In [16]:
sr_parser = nltk.ShiftReduceParser(grammar1, trace=2) #以跟踪模式运行上述分析器
sent = "Mary saw a dog".split()
# print(sr_parser.parse(sent))
for t in sr_parser.parse(sent):
    print(t)

Parsing 'Mary saw a dog'
    [ * Mary saw a dog]
  S [ 'Mary' * saw a dog]
  R [ NP * saw a dog]
  S [ NP 'saw' * a dog]
  R [ NP V * a dog]
  S [ NP V 'a' * dog]
  R [ NP V Det * dog]
  S [ NP V Det 'dog' * ]
  R [ NP V Det N * ]
  R [ NP V NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP Mary) (VP (V saw) (NP (Det a) (N dog))))


移进-规约分析器可能会到达一个死胡同，而不能找到任何解析，即使输入的句子是符合语法的。这种情况发生时，没有剩余的输入，而堆栈包含不能被规约到一个 S的项目。 问题出现的原因是：较早前做出的选择不能被分析器撤销（虽然图形演示中用户可以撤消它 们的选择）。分析器可以做两种选择：（a）当有多种规约可能时选择哪个规约，（b）当移进和规约都可以时选择哪个动作。 

移进-规约分析器可以改进执行策略来解决这些冲突。例如：它可以通过只有在不能规 约时才移进，解决移进-规约冲突；它可以通过优先执行规约操作，解决规约-规约冲突；它可以从堆栈移除更多的项目。（一个通用的移进-规约分析器，是一个“超前 LR 分析器”， 普遍使用在编程语言编译器中。） 

移进-规约分析器相比递归下降分析器的好处是，它们只建立与输入中的词对应的结构。 此外，每个结构它们只建立一次。例如：NP(Det(the), N(man))只建立和压入栈一次， 不管以后VP -> V NP PP 规约或者 NP -> NP PP 规约会不会用到。


#### 左角落分析器

递归下降分析器的问题之一是当它遇到一个左递归产生式时，会进入无限循环。这是因 为它盲目应用文法产生式而不考虑实际输入的句子。左角落分析器是我们已经看到的自下而 上与自上而下方法的混合体。 

**左角落分析器是一个带自下而上过滤的自上而下的分析器**。不像普通的递归下降分析器，它不会陷入左递归产生式的陷井。在开始工作之前，左角落分析器预处理上下文无关文法建立一个表，其中每行包含两个单元，第一个存放非终结符，第二个存放那个非终结符可能的左角落的集合。表 8-2 用 grammar2的文法演示了这一点。
![list8-2](./imgs/list8-2.jpg)

分析器每次考虑产生式时，它会检查下一个输入词是否与左角落表格中至少一种非终结符的类别相容。

#### 符合语句规则的子串表

上面讨论的简单的分析器在完整性和效率上都有限制。为了弥补这些，我们将运用**动态规划**算法设计技术分析问题。正如我们在 4.7节中所看到的，动态规划存储中间结果，并在适当的时候重用它们，能显著提高效率。

这种技术可以应用到句法分析，使我们能够存储分析任务的部分解决方案，然后在必要的时候查找它们，直到达到最终解决方案。这种分析方法被称为**图表分析**。我们在本节中介绍它的主要思想；更多的实施细节请看网上提供的本章的材料。

动态规划使我们能够只建立一次 `PP in my pajamas`。第一次我们建立时就把它存入一个表格中，然后在我们需要作为对象 `NP`或更高的`VP`的组成部分用到它时我们就查找表格。这个表格被称为**符合语法规则的子串表**或简称为**WFST**。（术语“串”指一个句子中的连续的词序列。）我们将展示如何自下而上的构建WFST，以便系统地记录已经找到的句法成分。 让我们设置我们的输入为句子（2）。指定`WFST` 的跨度的数值让人联想起 Python的分片符号（3.2节）。这种数据结构的另一种方式如图 8-6 所示，一个称为图表的数据结构。
![8-6](./imgs/8-6.jpg)

在 `WFST` 中，我们通过填充三角矩阵中的单元记录词的位置：纵轴表示一个子串的起始位置，而横轴表示结束位置（从而 `shot` 将出现在坐标`(1, 2)`的单元中）。为了简化这个演示，我们将假定每个词有一个独特的词汇类别，我们将在矩阵中存储词汇类别（不是词）。 所以单元`(1, 2)`将包含条目`V`。更一般的，如果我们输入的字符串是 `a1a2 ... an`，我们的文法包含一个形为`A → ai`的产生式，那么我们把`A` 添加到单元`(i-1, i)`。 所以，对于 `text` 中的每个词，我们可以在我们的文法中查找它所属的类别。

In [17]:
text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

对于我们的`WFST`，我们用 Python 中的**链表的链表**创建一个`(n-1)×(n-1)`的矩阵，在例8-3中的函数 `init_wfst()`中用每个标识符的词汇类型初始化它。我们还定义一个实用的函数 `display()`来为我们精美的输出`WFST`。正如预期的那样，`V`在`(1, 2)`单元中。

In [25]:
# 例8-3. 使用符合语句规则的子串表的接收器
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens + 1)] for j in range(numtokens + 1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i + 1] = productions[0].lhs()
    return wfst

def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1, nt2) in index:
                    wfst[start][end] = index[(nt1, nt2)]
                    if trace:
                        print ("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" %
                        (start, nt1, mid, nt2, end, start, index[(nt1, nt2)], end))
    return wfst

def display(wfst, tokens):
    print('\nWFST ' + ' '.join([("%-4d" % i) for i in range(1, len(wfst))]))
    for i in range(len(wfst) - 1):
        print("%d   " % i, end="")
        for j in range(1, len(wfst)): 
            print("%-4s" % (wfst[i][j] or '.'), end="")
        print()

tokens = "I shot an elephant in my pajamas".split()
wfst0 = init_wfst(tokens, groucho_grammar)
display(wfst0, tokens)


WFST 1    2    3    4    5    6    7   
0   NP  .   .   .   .   .   .   
1   .   V   .   .   .   .   .   
2   .   .   Det .   .   .   .   
3   .   .   .   N   .   .   .   
4   .   .   .   .   P   .   .   
5   .   .   .   .   .   Det .   
6   .   .   .   .   .   .   N   


In [24]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)
display(wfst1, tokens)


WFST 1    2    3    4    5    6    7   
0    NP  .   .   S   .   .   S   
1    .   V   .   VP  .   .   VP  
2    .   .   Det NP  .   .   .   
3    .   .   .   N   .   .   .   
4    .   .   .   .   P   .   PP  
5    .   .   .   .   .   Det NP  
6    .   .   .   .   .   .   N   


回到我们的表格表示，假设对于词 `an` 我们有 `Det` 在`(2, 3)`单元，对以词 `elephant` 有 `N`
在`(3, 4)`单元，对于`an elephant` 我们应该在`(2, 4)`放入什么？我们需要找到一个形如`NP → Det N`的产生式。查询了文法，我们知道我们可以输入`(0, 2)`单元的 `NP`。

更一般的，我们可以在`(i, j)`输入 `A`，如果有一个产生式`A → B C`，并且我们在`(i, k)` 中找到非终结符 `B`，在`(k, j)`中找到非终结符`C`。例 8-3中的程序使用此规则完成`WFST`。通过调用函数`complete_wfst()`时设置 `trace` 为 `True`，我们看到了显示`WFST` 正在被创建的跟踪输出

In [26]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)

[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]


我们得出结论：只要我们已经在(0, 7)单元构建了一个 `S` 节点，表明我们已经找到了一
个涵盖了整个输入的句子，我们就为整个输入字符串找到了一个解析。最后的WFST 状态 如图 8-7 所示。
![8-77](./imgs/8-77.jpg)

请注意，在这里我们没有使用任何内置的分析函数。我们已经从零开始实现了一个完整的**初级图表分析器**！ 

`WFST` 有几个缺点。首先，正如你可以看到的，`WFST` 本身不是一个分析树，所以该技术严格地说是认识到一个句子被一个文法承认，而不是分析它。其次，它要求每个非词汇文法生产式是二元的。虽然可以将任意的 `CFG` 转换为这种形式，我们宁愿使用这种方法时没有这样的规定。第三，作为一个自下而上的方法，它潜在的存在浪费，它会在不符合文法的地方提出成分。 最后，WFST 并不能表示句子中的结构歧义（如两个动词短语的读取）。`(2, 8)`单元中的 `VP` 实际上被输入了两次，一次是读取`V NP`，一次是读取`VP PP`。这是不同的假设，第二个会覆盖第一个（虽然如此，这并不重要，因为左侧是相同的。）图表分析器使用稍微更丰富的数据结构和一些有趣的算法来解决这些问题（见 8.8节）。

In [28]:
nltk.app.chartparser()

grammar= (
('    ', 'S -> NP VP,')
('    ', 'VP -> VP PP,')
('    ', 'VP -> V NP,')
('    ', 'VP -> V,')
('    ', 'NP -> Det N,')
('    ', 'NP -> NP PP,')
('    ', 'PP -> P NP,')
('    ', "NP -> 'John',")
('    ', "NP -> 'I',")
('    ', "Det -> 'the',")
('    ', "Det -> 'my',")
('    ', "Det -> 'a',")
('    ', "N -> 'dog',")
('    ', "N -> 'cookie',")
('    ', "N -> 'table',")
('    ', "N -> 'cake',")
('    ', "N -> 'fork',")
('    ', "V -> 'ate',")
('    ', "V -> 'saw',")
('    ', "P -> 'on',")
('    ', "P -> 'under',")
('    ', "P -> 'with',")
)
tokens = ['John', 'ate', 'the', 'cake', 'on', 'the', 'table']
Calling "ChartParserApp(grammar, tokens)"...


### 8.5 依存关系和依存文法

短语结构文法是关于词和词序列如何结合起来形成句子成分的。一个独特的和互补的方式，**依存文法**，集中关注的是**词与其他词之间的关系**。依存关系是**一个中心词与它的依赖之间的二元对称关系**。一个句子的中心词通常是动词，所有其他词要么依赖于中心词，要么依赖路径与它联通。

**依存关系表示是一个加标签的有向图**，其中**节点是词汇项**，**加标签的弧表示依赖关系**， 从中心词到依赖。图 8-8 显示了一个依存关系图，箭头从中心词指出它们的依赖。
![8-8](./imgs/8-8.jpg)

图 8-8 中的弧加了依赖与它的中心词之间的语法功能标签。例如：`I` 是 `shot`（这是整个
句子的中心词）的 `SBJ`（主语），`in` 是一个 `NMOD`（`elephant` 的名词修饰语）。与短语结构文 法相比，依存文法可以作为一种依存关系直接用来表示语法功能。 

下面是 NLTK为依存文法编码的一种方式——注意它只能捕捉依存关系信息，不能指定依存关系类型：

In [33]:
# groucho_dep_grammar = nltk.parse_dependency_grammar("""
groucho_dep_grammar = nltk.grammar.DependencyGrammar.fromstring("""
'shot' -> 'I' | 'elephant' | 'in'
'elephant' -> 'an' | 'in' 
'in' -> 'pajamas' 
'pajamas' -> 'my'
""")
print(groucho_dep_grammar)

Dependency grammar with 7 productions
  'shot' -> 'I'
  'shot' -> 'elephant'
  'shot' -> 'in'
  'elephant' -> 'an'
  'elephant' -> 'in'
  'in' -> 'pajamas'
  'pajamas' -> 'my'


依存关系图是一个**投影**，当所有的词都按线性顺序书写，边可以在词上绘制而不会交叉。这等于是说一个词及其所有后代依赖（依赖及其依赖的依赖，等等）在句子中形成一个连续的词序列。图 8-8是一个投影，我们可以使用**投影依存关系分析器**分析很多英语句子。下面的例子演示 `groucho_dep_grammar` 如何提供了一种替代的方法来捕捉附着歧义，我们之前在研究短语结构语法中遇到的。

In [34]:
pdp = nltk.ProjectiveDependencyParser(groucho_dep_grammar)
sent = 'I shot an elephant in my pajamas'.split()
trees = pdp.parse(sent)
for tree in trees:
    print(tree)

(shot I (elephant an (in (pajamas my))))
(shot I (elephant an) (in (pajamas my)))


#### 配价与词汇

在依存文法的传统中，在表 8-3 中的动词被认为具有不同的配价。配价限制不仅适用于动词，也适用于其他类的中心词。

![list8-3](./imgs/list8-3.jpg)

配价是一个词项的属性，我们将在第 9 章进一步讨论它。补语往往与修饰语对照，虽然两者都是依存关系的类别。


### 8.6 文法开发

#### 树库和文法
`corpus`模块定义了树库语料的阅读器，其中包含了宾州树库语料的 10％的样本。

In [35]:
from nltk.corpus import treebank
t = treebank.parsed_sents("wsj_0001.mrg")[0]
print(t)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))


我们可以利用这些数据来帮助开发一个文法。例如：例 8-4中的程序使用一个简单的过滤器找出带句子补语的动词。假设我们已经有一个形如 `VP -> SV S` 的产生式，这个信息使我们能够识别那些包括在`SV` 的扩张中的特别的动词。

In [37]:
# 例8-4. 搜索树库找出句子的补语。
def filter(tree):
    child_nodes = [child.label() for child in tree if isinstance(child, nltk.Tree)]
    return (tree.label() == 'VP') and ('S' in child_nodes)

from nltk.corpus import treebank
[subtree for tree in treebank.parsed_sents()
    for subtree in tree.subtrees(filter)]

[Tree('VP', [Tree('VBN', ['named']), Tree('S', [Tree('NP-SBJ', [Tree('-NONE-', ['*-1'])]), Tree('NP-PRD', [Tree('NP', [Tree('DT', ['a']), Tree('JJ', ['nonexecutive']), Tree('NN', ['director'])]), Tree('PP', [Tree('IN', ['of']), Tree('NP', [Tree('DT', ['this']), Tree('JJ', ['British']), Tree('JJ', ['industrial']), Tree('NN', ['conglomerate'])])])])])]),
 Tree('VP', [Tree('VBD', ['said']), Tree(',', [',']), Tree('``', ['``']), Tree('S', [Tree('NP-SBJ', [Tree('DT', ['This'])]), Tree('VP', [Tree('VBZ', ['is']), Tree('NP-PRD', [Tree('DT', ['an']), Tree('JJ', ['old']), Tree('NN', ['story'])])])])]),
 Tree('VP', [Tree('VBD', ['said']), Tree('S', [Tree('-NONE-', ['*T*-1'])])]),
 Tree('VP', [Tree('VBN', ['expected']), Tree('S', [Tree('-NONE-', ['*?*'])])]),
 Tree('VP', [Tree('VBD', ['said']), Tree('S', [Tree('-NONE-', ['*T*-1'])])]),
 Tree('VP', [Tree('VBZ', ['appears']), Tree('S', [Tree('NP-SBJ', [Tree('-NONE-', ['*-1'])]), Tree('VP', [Tree('TO', ['to']), Tree('VP', [Tree('VB', ['be']), Tree('

`PP`附着语料库，`nltk.corpus.ppattach`，是另一个有关特别动词配价的信息源。在这里，我们演示挖掘这个语料库的技术。它找出具有固定的介词和名词的介词短语对，其中介词短语附着到`VP` 还是`NP`，由选择的动词决定。

In [38]:
entries = nltk.corpus.ppattach.attachments('training')
table = nltk.defaultdict(lambda: nltk.defaultdict(set))
for entry in entries:
    key = entry.noun1 + '-' + entry.prep + '-' + entry.noun2
    table[key][entry.attachment].add(entry.verb)

In [39]:
for key in sorted(table):
    if len(table[key]) > 1:
        print(key, 'N:', sorted(table[key]['N']), 'V:', sorted(table[key]['V']))

%-below-level N: ['left'] V: ['be']
%-from-year N: ['was'] V: ['declined', 'dropped', 'fell', 'grew', 'increased', 'plunged', 'rose', 'was']
%-in-August N: ['was'] V: ['climbed', 'fell', 'leaping', 'rising', 'rose']
%-in-September N: ['increased'] V: ['climbed', 'declined', 'dropped', 'edged', 'fell', 'grew', 'plunged', 'rose', 'slipped']
%-in-week N: ['declined'] V: ['was']
%-to-% N: ['add', 'added', 'backed', 'be', 'cut', 'go', 'grow', 'increased', 'increasing', 'is', 'offer', 'plummet', 'reduce', 'rejected', 'rise', 'risen', 'shaved', 'wants', 'yield', 'zapping'] V: ['fell', 'rise', 'slipped']
%-to-million N: ['declining'] V: ['advanced', 'climbed', 'cutting', 'declined', 'declining', 'dived', 'dropped', 'edged', 'fell', 'gained', 'grew', 'increased', 'jump', 'jumped', 'plunged', 'rising', 'rose', 'slid', 'slipped', 'soared', 'tumbled']
1-to-21 N: ['dropped'] V: ['dropped']
1-to-33 N: ['gained'] V: ['dropped', 'fell', 'jumped']
1-to-4 N: ['added'] V: ['gained']
1-to-47 N: ['jumped']

NLTK 语料库也收集了中央研究院树库语料，包括10000 句已分析的句子，来自现代汉语中央研究院平衡语料库。让我们加载并显示这个语料库中的一棵树。

In [40]:
nltk.corpus.sinica_treebank.parsed_sents()[3450].draw()

#### 有害的歧义
#### 加权文法

### 8.7 小结

- 句子都有内部组织结构，可以用一棵树表示。组成结构的显著特点是：递归、中心词、补语和修饰语。
- 文法是一个潜在的无限的句子集合的一个紧凑的特性；我们说，一棵树是符合语法规则的或文法树授权一棵树。
- 文法是用于描述一个给定的短语是否可以被分配一个特定的成分或依赖结构的一种形式化模型。
- 给定一组句法类别，上下文无关文法使用一组生产式表示某类型`A` 的短语如何能够被分析成较小的序列`α1 ... αn`。
- 依存文法使用产生式指定给定的中心词的依赖是什么。
- 当一个句子有一个以上的文法分析就产生句法歧义（如介词短语附着歧义）
- 分析器是一个过程，为符合语法规则的句子寻找一个或多个相应的树。
- 一个简单的自上而下分析器是递归下降分析器，在文法产生式的帮助下递归扩展开始符号（通常是`S`），尝试匹配输入的句子。这个分析器并不能处理左递归产生式（如：`NP -> NP PP`）。它盲目扩充类别而不检查它们是否与输入字符串兼容的方式效率低下，而且会重复扩充同样的非终结符然后丢弃结果。
- 一个简单的自下而上的分析器是移位-规约分析器，它把输入移到一个堆栈中，并尝试匹配堆栈顶部的项目和文法产生式右边的部分。这个分析器不能保证为输入找到一个有 效的解析，即使它确实存在，它建立子结构而不检查它是否与全部文法一致。